In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import json
import contextily as ctx
import warnings
warnings.filterwarnings("ignore")

In [5]:
LBCS = {
    'SITIO ERIAZO': 9000,
    'BIENES COMUNES': 9999,
    'OTROS': 9200,
    'SALUD': 6500,
    'CULTO': 6600,
    'BODEGA Y ALMACENAJE': 3120,
    'HABITACIONAL': 1100,
    'OFICINA': 2300,
    'COMERCIO': 2100,
    'EDUCACION Y CULTURA': 4100,
    'INDUSTRIA': 3000,
    'AREAS VERDES': 7000,
    'PRE ROL': 9999,
    'TRANSPORTE Y TELEC': 4000, #434public0 Telecomunicaciones
    'DEPORTE Y RECREACION': 7100,
    'HOTEL, MOTEL': 1200,
    'ESTACIONAMIENTO': 5210,
    'ADM PUBLICA Y DEFENSA': 6200, #+6300
    'MULTI ROL': 9999,
    'AGRICOLA': 9000,
}

LBCS_translated = {
    'SITIO ERIAZO': 'Vacant lot',
    'BIENES COMUNES': 'Public spaces',
    'OTROS': 'Other',
    'SALUD': 'Healthcare',
    'CULTO': 'Religious',
    'BODEGA Y ALMACENAJE': 'Warehouse and storage',
    'HABITACIONAL': 'Residential',
    'OFICINA': 'Office',
    'COMERCIO': 'Commercial',
    'EDUCACION Y CULTURA': 'Education and culture',
    'INDUSTRIA': 'Industry',
    'AREAS VERDES': 'Park',
    'TRANSPORTE Y TELEC': 'Transportation and telecommunications', #4340 Telecommunications
    'DEPORTE Y RECREACION': 'Sports and recreation',
    'HOTEL, MOTEL': 'Hotel, motel',
    'ESTACIONAMIENTO': 'Parking',
    'ADM PUBLICA Y DEFENSA': 'Public administration and defense', #+6300
    'AGRICOLA': 'Agricultural',
}

In [17]:
grid = gpd.read_parquet('/app/data/new_grid_cityscope.parquet')
grid['name'] = grid['name'].map(LBCS_translated)
grid['interactive'] = False

In [18]:
def read_grid_geojson(filename):    
    with open(filename, 'r') as file:
        grid_dict = json.load(file)
        return grid_dict

def extract_types_gdf(filename):
    grid_dict = read_grid_geojson(filename)
    types = pd.DataFrame.from_dict(grid_dict['GEOGRID']['properties']['types']).transpose()
    return types

grid_filename = '/app/data/grid.geojson'

types = extract_types_gdf(grid_filename)

In [19]:
types.reset_index()

,index,id,name,description,color,height,height[0],height[1],height[2],LBCS,NAICS,interactive
0,Office,0,Office,"Offices and other commercial buildings, 0-100 ...",#2482c6,"[0, 50, 100]",0,50,100,"[{'proportion': 1, 'use': {'2310': 1}}]","[{'proportion': 1, 'use': {'5400': 1}}]",True
1,Campus,1,Campus,"Campus buildings, non-interactive, 0-30 stories",#ab8f39,"[0, 15, 30]",0,15,30,"[{'proportion': 1, 'use': {'2310': 1}}]","[{'proportion': 1, 'use': {'5400': 1}}]",False
2,Park,2,Park,"Parks, playgrounds, and other open spaces. No ...",#7eb346,"[0, 0, 0]",0,0,0,"[{'proportion': 1, 'use': {'7240': 1}}]",None,True
3,Residential,3,Residential,"Residential buildings and apartments, 0-100 st...",#b97e18,"[0, 50, 100]",0,50,100,"[{'proportion': 1, 'use': {'1100': 1}}]",None,True


In [50]:
new_types = pd.DataFrame(columns=types.columns)
new_types.drop(columns=['color'], inplace=True)

In [51]:
new_names = grid['name'].unique()

In [52]:
new_types['index'] = new_names
new_types['name'] = new_names
new_types['description'] = ''
new_types['interactive'] = False

In [53]:
# Eliminar duplicados en 'grid' basados en 'name'
grid_unique = grid.drop_duplicates(subset='name')

# Realizar el cruce para añadir la columna 'color' desde 'grid_unique' a 'new_types'
new_types = new_types.merge(grid_unique[['name', 'color']], on='name', how='left')

In [58]:
new_types

,id,name,description,height,height[0],height[1],height[2],LBCS,NAICS,interactive,index,color
0,NaN,Vacant lot,,NaN,NaN,NaN,NaN,NaN,NaN,False,Vacant lot,"[229, 115, 115]"
1,NaN,Warehouse and storage,,NaN,NaN,NaN,NaN,NaN,NaN,False,Warehouse and storage,"[240, 98, 146]"
2,NaN,Residential,,NaN,NaN,NaN,NaN,NaN,NaN,False,Residential,"[129, 199, 132]"
3,NaN,Religious,,NaN,NaN,NaN,NaN,NaN,NaN,False,Religious,"[149, 117, 205]"
4,NaN,Office,,NaN,NaN,NaN,NaN,NaN,NaN,False,Office,"[174, 213, 129]"
5,NaN,Commercial,,NaN,NaN,NaN,NaN,NaN,NaN,False,Commercial,"[244, 143, 177]"
6,NaN,Public spaces,,NaN,NaN,NaN,NaN,NaN,NaN,False,Public spaces,"[220, 231, 117]"
7,NaN,Healthcare,,NaN,NaN,NaN,NaN,NaN,NaN,False,Healthcare,"[255, 183, 77]"
8,NaN,Other,,NaN,NaN,NaN,NaN,NaN,NaN,False,Other,"[77, 208, 225]"
9,NaN,Education and culture,,NaN,NaN,NaN,NaN,NaN,NaN,False,Education and culture,"[100, 181, 246]"
